In [1]:
# Imports

# File handling
import os
import pathlib
from zipfile import ZipFile
import splitfolders

# Visualization
import  matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Data and maths
import numpy as np
import pandas as pd
import random

# ML
import tensorflow as tf
from tensorflow import keras
from keras import models, layers
from keras.utils import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
#from keras.callbacks import CSVLogger # if problems with SAVING model

# Metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [19]:
# image data
splitfolders.ratio("data/images_unf/", output="images", seed=586, ratio=(.8, .1, .1), group_prefix=None, move=True)

Copying files: 21165 files [01:16, 277.95 files/s]
Copying files: 0 files [42:48, ? files/s]

In [5]:
# preprocess train, test and valid sets
train_path = 'images/train/'
test_path = 'images/test/'
valid_path = 'images/val/'
size = (256, 256)

train = image_dataset_from_directory(directory=train_path,
                         labels='inferred',
                         color_mode="rgb",
                         batch_size=64,
                         image_size=size,
                         shuffle=True,
                         seed=77)

test = image_dataset_from_directory(directory=test_path,
                         labels='inferred',
                         color_mode="rgb",
                         batch_size=64,
                         image_size=size,
                         shuffle=False,
                         seed=77)

valid = image_dataset_from_directory(directory=valid_path,
                         labels='inferred',
                         color_mode="rgb",
                         batch_size=64,
                         image_size=size,
                         shuffle=False,
                         seed=77)

Found 16930 files belonging to 2 classes.
Found 2120 files belonging to 2 classes.
Found 2115 files belonging to 2 classes.


model 1: loss: 0.6931 - accuracy: 0.4816 - val_loss: 0.6931 - val_accuracy: 0.4818  

model 2: loss: 8.3040 - accuracy: 0.4834 (interrupted)  

model 3: loss: 7.7412 - accuracy: 0.5178 - val_loss: 7.7656 - val_accuracy: 0.5182 (interrupted)  

model 4: loss: 0.7825 - accuracy: 0.4816 - val_loss: 0.6931 - val_accuracy: 0.4818  

model 5: loss: 0.4360 - accuracy: 0.8014 - val_loss: 0.4345 - val_accuracy: 0.8052  

model 6: loss: 0.4879 - accuracy: 0.7790 - val_loss: 0.4714 - val_accuracy: 0.7721  

model 7: loss: 0.3973 - accuracy: 0.8236 - val_loss: 0.4227 - val_accuracy: 0.8132  

model 8: loss: 0.0499 - accuracy: 0.9820 - val_loss: 0.6409 - val_accuracy: 0.8771  


In [3]:
input_shape = (256, 256, 1)
num_classes=2

model1 = keras.Sequential(
    [
        layers.Rescaling(1./255, input_shape=input_shape),
                         
        layers.Conv2D(16, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(pool_size=(2, 2)),
                         
        layers.Conv2D(32, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        layers.Conv2D(64, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        layers.Flatten(), # Dimensionality reduction (feature map)
        layers.Dropout(0.5), # Output layer
        layers.Dense(num_classes) # fully connected layer
    ])

model1.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 16)      160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 64)        1

In [28]:
input_shape = (256, 256, 1)
num_classes=2

model2 = keras.Sequential(
    [
        layers.Rescaling(1./255, input_shape=input_shape),
                         
        layers.Conv2D(32, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(),
        
        layers.Conv2D(64, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(),
        
        layers.Flatten(), # Dimensionality reduction (feature map)
        layers.Dropout(0.5), # Output layer
        layers.Dense(num_classes) # fully connected layer
    ])

model2.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model2.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_10 (Rescaling)    (None, 256, 256, 1)       0         
                                                                 
 conv2d_20 (Conv2D)          (None, 254, 254, 32)      320       
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 127, 127, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_21 (MaxPoolin  (None, 62, 62, 64)       0         
 g2D)                                                            
                                                                 
 flatten_7 (Flatten)         (None, 246016)           

In [39]:
input_shape = (256, 256, 1)
num_classes=2

model3 = keras.Sequential(
    [
        layers.Rescaling(1./255, input_shape=input_shape),
                         
        layers.Conv2D(32, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(),
        
        layers.Conv2D(64, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(),
        
        layers.Flatten(), # Dimensionality reduction (feature map)
        layers.Dropout(0.5), # Output layer
        layers.Dense(num_classes) # fully connected layer
    ])

model3.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model3.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_14 (Rescaling)    (None, 256, 256, 1)       0         
                                                                 
 conv2d_28 (Conv2D)          (None, 254, 254, 32)      320       
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 127, 127, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_29 (MaxPoolin  (None, 62, 62, 64)       0         
 g2D)                                                            
                                                                 
 flatten_11 (Flatten)        (None, 246016)          

In [43]:
input_shape = (256, 256, 1)
num_classes=2

model4 = keras.Sequential(
    [
        layers.Rescaling(1./255, input_shape=input_shape),
                         
        layers.Conv2D(32, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(),
        
        layers.Conv2D(64, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(),
        
        layers.Conv2D(128, kernel_size=(3,3), activation="relu"), # conv layer
        layers.MaxPooling2D(),
        
        layers.Flatten(), # Dimensionality reduction (feature map)
        layers.Dropout(0.5), # Output layer
        layers.Dense(num_classes) # fully connected layer
    ])

model4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model4.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_16 (Rescaling)    (None, 256, 256, 1)       0         
                                                                 
 conv2d_32 (Conv2D)          (None, 254, 254, 32)      320       
                                                                 
 max_pooling2d_32 (MaxPoolin  (None, 127, 127, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_33 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 62, 62, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_34 (Conv2D)          (None, 60, 60, 128)     

In [56]:
shape = (256, 256, 3)
num_classes=2

model5 = keras.Sequential(
    [
        layers.Rescaling(1./255, input_shape=shape), # input
        layers.Flatten(),
        
        layers.Dense(300, activation="relu"), # layer
        layers.Flatten(),
        
        layers.Dense(100, activation="relu"), # layer
        layers.Flatten(),
        
        layers.Dense(num_classes, activation="sigmoid") # output
    ])

model5.compile(optimizer='SGD',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model5.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_13 (Rescaling)    (None, 256, 256, 3)       0         
                                                                 
 flatten_13 (Flatten)        (None, 196608)            0         
                                                                 
 dense_45 (Dense)            (None, 300)               58982700  
                                                                 
 flatten_14 (Flatten)        (None, 300)               0         
                                                                 
 dense_46 (Dense)            (None, 100)               30100     
                                                                 
 flatten_15 (Flatten)        (None, 100)               0         
                                                                 
 dense_47 (Dense)            (None, 2)               

In [60]:
shape = (256, 256, 3)
num_classes=2

model6 = keras.Sequential(
    [
        layers.Rescaling(1./255, input_shape=shape), # input
        layers.Flatten(),
        
        layers.Dense(300, activation="relu"), # layer
        layers.Flatten(),
        
        layers.Dense(100, activation="relu"), # layer
        layers.Flatten(),
        
        layers.Dense(num_classes, activation="softmax") # output
    ])

model6.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model6.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_14 (Rescaling)    (None, 256, 256, 3)       0         
                                                                 
 flatten_16 (Flatten)        (None, 196608)            0         
                                                                 
 dense_48 (Dense)            (None, 300)               58982700  
                                                                 
 flatten_17 (Flatten)        (None, 300)               0         
                                                                 
 dense_49 (Dense)            (None, 100)               30100     
                                                                 
 flatten_18 (Flatten)        (None, 100)               0         
                                                                 
 dense_50 (Dense)            (None, 2)               

In [ ]:
shape = (256, 256, 3)
num_classes=2

model7 = keras.Sequential(
    [   
        layers.Rescaling(1./255, input_shape=shape), # input
        layers.Flatten(),
        
        layers.Dense(200, activation="relu"), # layer
        layers.Flatten(),
        
        layers.Dense(100, activation="relu"), # layer
        layers.Flatten(),

        layers.Dense(100, activation="relu"), # layer
        layers.Flatten(),
        
        layers.Dense(num_classes, activation="softmax") # output
    ])

model7.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [6]:
shape = (256, 256, 3)
num_classes=2

model8 = keras.Sequential(
    [   
        layers.Rescaling(1./255, input_shape=shape), # input
        layers.MaxPooling2D(2),
     
        layers.Conv2D(200, kernel_size=(3, 3), activation="relu"), # layer
        layers.MaxPooling2D(2),
        
        layers.Conv2D(150, kernel_size=(3, 3), activation="relu"), # layer
        layers.MaxPooling2D(2),

        layers.Conv2D(50, kernel_size=(3, 3), activation="relu"), # layer
        layers.MaxPooling2D(2),
        layers.Flatten(),
        
        layers.Dense(num_classes, activation="softmax") # output
    ])

model8.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
model8 = model8.fit(train, validation_data=valid, verbose=1, epochs=25)

Epoch 1/25
265/265 [==============================] - 549s 2s/step - loss: 0.5114 - accuracy: 0.7470 - val_loss: 0.4393 - val_accuracy: 0.7953
Epoch 2/25
265/265 [==============================] - 517s 2s/step - loss: 0.4171 - accuracy: 0.8146 - val_loss: 0.3773 - val_accuracy: 0.8336
Epoch 3/25
265/265 [==============================] - 524s 2s/step - loss: 0.3902 - accuracy: 0.8312 - val_loss: 0.4130 - val_accuracy: 0.8132
Epoch 4/25
237/265 [=========================>....] - ETA: 55s - loss: 0.3617 - accuracy: 0.8452

KeyboardInterrupt: 

In [ ]:
shape = (256, 256, 3)
num_classes=2

model9 = keras.Sequential(
    [   
        layers.Rescaling(1./255, input_shape=shape), # input
        layers.MaxPooling2D(pool_size=(2, 2)),
     
        layers.Conv2D(256, kernel_size=(2, 2), activation="relu"), # layer
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        layers.Conv2D(64, kernel_size=(2, 2), activation="relu"), # layer
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(32, kernel_size=(2, 2), activation="relu"), # layer
        layers.MaxPooling2D(pool_size=(2, 2)),

     
        layers.Flatten(),
        layers.Dropout(0.4),     
        layers.Dense(num_classes, activation="softmax") # output
    ])

model9.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])